In [1]:
from pathlib import Path
import xml.etree.ElementTree as ET

In [2]:
_CORPORA = Path("/media/storage/jim/speech-syn/Corpora/")

In [3]:
_ANB = _CORPORA / "ga_UL" / "anb"

In [4]:
_ANB_DIRS_GLOBBED = _ANB.glob("*")

In [8]:
# text01-001 is a duplicate of text01 without corrections
# new_text01 has no xml directory
_SKIP = ["text01-001", "new_text01"]
_ANB_DIRS = [a for a in _ANB_DIRS_GLOBBED if a.stem not in _SKIP]

In [9]:
def get_words(file):
    words = []
    junk = ["GLOTTAL_STOP", "SILENCE_TOKEN"]
    tree = ET.parse(file)
    root = tree.getroot()
    for wordtag in root.findall(".//word"):
        if "pos" in wordtag.attrib and wordtag.attrib["pos"] == "punc":
            continue
        if "input_string" in wordtag.attrib and wordtag.attrib["input_string"] not in junk:
            words.append(wordtag.attrib["input_string"])
    return words

In [10]:
texts = {}
for dir in _ANB_DIRS:
    xmldir = dir / "xml"
    for xmlfile in xmldir.glob("*.xml"):
        texts[xmlfile.stem] = get_words(xmlfile)

In [11]:
_ANB_OUTDIR = "/media/storage/jim/Playing/mfa-corp/anb/"

In [12]:
for outname in texts.keys():
    with open(f"{_ANB_OUTDIR}{outname}.txt", "w") as outf:
        outf.write(" ".join(texts[outname]))

In [13]:
def make_dict(file):
    words = []
    junk = ["GLOTTAL_STOP", "SILENCE_TOKEN"]
    tree = ET.parse(file)
    root = tree.getroot()
    for wordtag in root.findall(".//word"):
        if "pos" in wordtag.attrib and wordtag.attrib["pos"] == "punc":
            continue
        if "input_string" in wordtag.attrib and wordtag.attrib["input_string"] not in junk:
            word = wordtag.attrib["input_string"]
            phones = []
            for phoneme in wordtag.findall(".//phoneme"):
                if "symbol" in phoneme.attrib:
                    phones.append(phoneme.attrib["symbol"])
            if len(phones) != 0:
                words.append(" ".join([word] + phones))
    return words

In [14]:
dictionary = set()
for dir in _ANB_DIRS:
    xmldir = dir / "xml"
    for xmlfile in xmldir.glob("*.xml"):
        dictionary.update(make_dict(xmlfile))

In [15]:
_ANB_DICT = "/media/storage/jim/Playing/mfa-corp/anb.dict"
with open(_ANB_DICT, "w") as dictf:
    for line in sorted(dictionary):
        dictf.write(line + "\n")

In [ ]:
_MFA_OUT = "/media/storage/jim/Playing/mfa_alignments/anb"
_MFA_MODEL = "/media/storage/jim/Playing/mfa_models/ga_UL"

In [ ]:
!mfa train -o {_MFA_MODEL} {_PMG_OUTDIR} {_PMG_DICT} {_MFA_OUT}